# Parte 1 – Conjunto de Datos

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.utils.random import sample_without_replacement
import os
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

In [2]:
TRAIN_DATA_PATH = "/kaggle/input/amex-default-prediction/train_data.csv"
TRAIN_LABELS_PATH = "/kaggle/input/amex-default-prediction/train_labels.csv"
TEST_DATA_PATH = "/kaggle/input/amex-default-prediction/test_data.csv"
SAMPLE_SUBMISSION_PATH = "/kaggle/input/amex-default-prediction/sample_submission.csv"

Dado que el archivo **train_data.csv** es muy grande y que la plataforma del notebook ofrece una memoria limitada, vamos a leer el archivo en chunks de tamaño 100.000 .

Luego, para el sampleo utilizaremos el archivo **train_labels.csv** el cual es considerablemente pequeño y contiene el ID de cada cliente junto con su target.

Luego guardamos en un dataframe los IDs de los clientes que vamos a utilizar.

In [3]:
df_train_total = pd.read_csv(TRAIN_DATA_PATH, chunksize=100000)
df_train_labels = pd.read_csv(TRAIN_LABELS_PATH)

numero_grupo = 2
filas = len(df_train_labels)
semilla = (31416 * numero_grupo)%1000
n_population = filas # cantidad de casos totales
n_samples = filas*0.05 # cantidad de casos que vamos a utilizar

sample = sample_without_replacement(random_state = semilla,
                                    n_population = n_population,
                                    n_samples = n_samples)
sample.sort()

df_sample_casos = df_train_labels.iloc[sample]

Una vez obtenidos los casos que vamos a utilizar, realizamos un inner join del dataframe con los IDs de los clientes y los chunks del archivo **train_data.csv**. Luego guardamos en un archivo csv el dataframe final.

In [4]:
def merge_data(data_to_use, total_data):
    df_train = pd.DataFrame()
    for chunk in total_data:
        data = pd.merge(data_to_use, chunk, left_on='customer_ID', right_on='customer_ID')
        df_train = df_train.append(data, ignore_index=True)
        del data
        del chunk
    return df_train

Debido a que el iterar por todos los chunks del dataframe tarda mucho, decidimos hacerlo una unica vez y simplemente cargar el dataframe a utilizar en un csv y cargarlo directamente con pandas.
> Warning: La siguiente celda tarda bastante

In [5]:
'''
df = merge_data(df_sample_casos, df_train_total)
df.to_csv('train_final.csv', sep=";")'''

Una vez tenemos el csv de train final, lo utilizamos para crear nuestro dataframe de entrenamiento.

In [6]:
df_train = pd.read_csv("https://drive.google.com/uc?export=download&id=171ytMl9FD_3ykCZwrZbI0iXlrZKuRYaS&confirm=t&uuid=4d6c6d66-735a-4be6-baca-fb65f980bcd8", sep=";")

# Parte 2 – Ciencia de Datos

## Análisis y exploración inicial

Empezamos viendo como quedo nuestro dataframe de entrenamiento

In [7]:
df_train.head()

Como vemos hay una columna no deseada llamada _Unnamed: 0_ es probable que sea producto de un error en el merge hecho anteriormente o el pasar el dataframe a un nuevo csv asi que vamos a borrarla

In [8]:
df_train.drop(columns = ["Unnamed: 0"], inplace = True)

Vamos a imprimir un poco de info para ver como estan compuestos nuestros datos

In [9]:
df_train.info(verbose = True)

Vamos a ver como estan distribuidos nuestros valores en la variable target

In [10]:
sns.set(rc={"figure.figsize":(9, 5)})
ax = sns.countplot(x="target", data=df_train)
ax.set_xticklabels(["Non-Default", "Default"])
plt.title("Cantidad de casos segun target")
plt.show()

Con un analisis inicial vemos que tenemos un dataset de entrenamiento algo desbalanceado, pero no parece lo suficiente como para intentar utilizar técnicas de balanceo en una primera instancia, de todos modos queda como una opción a considerar según los resultados que obtengamos con los modelos

### Sobre los datos faltantes o mal ingresados

Analizamos la existencia de NaNs en cada columna de nuestro dataset

In [11]:
sns.set(rc={"figure.figsize":(9, 5)})
df_train.isnull().any().value_counts().plot(kind='barh')
plt.title("Cantidad de features segun contengan o no valores nulos")
plt.show()

Observamos entonces que estamos trabajando con nans en más de la mitad de nuestras columnas, ahora queremos saber la proporción de los mismos en cada una.

In [12]:
background_color = 'white'
missing = pd.DataFrame(columns = ['% Missing Data'],data = df_train.isna().sum()/len(df_train))
missing = missing.sort_values(by=['% Missing Data'])
fig = plt.figure(figsize = (20, 60),facecolor=background_color)
gs = fig.add_gridspec(1, 2)
gs.update(wspace = 0.5, hspace = 0.5)
ax0 = fig.add_subplot(gs[0, 0])
for s in ["right", "top","bottom","left"]:
    ax0.spines[s].set_visible(False)
sns.heatmap(missing.loc[missing['% Missing Data'] > 0],cbar = False,annot = True,fmt =".2%", linewidths = 2,vmax = 1, ax = ax0)
plt.title("Porcentaje de NaNs por feature")
plt.show()

Hay muchas columnas con más de un 80% de nans, consideramos que estas columnas son fuertes candidatas a ser eliminadas ya que tienen su gran mayoria de datos faltantes y va a ser dificil realizar alguna tecnica de sustitucion para esas columnas ya que podrian ser poco representativas. Quizás para las que tienen menor porcentaje de nans se puede recurrir a alguna técnica para trabajar con los mismos.

Una suposicion que quizas se podría tener en cuenta en un análisis más profundo es que las columnas con D (Delinquency) tienen grandes cantidades de NaNs en sus valores y puede deberse a que si el cliente no cometio ningun delito o algo que se relaciona con esto se llena con NaN ese valor.

### Sobre los valores atípicos

Como dice la consigna y la descripcion dada en la competencia del dataset, presentamos muchas variables categoricas que tienen valores numericos. Como no conocemos que son estos valores decidimos no mapearlos a ninguna categoria en particular y trabajar con ellos directamente.

In [13]:
categoricas = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64' ,'D_66', 'D_68']
ignorar = ['S_2', 'customer_ID', 'target']

Creamos una funcion que nos permita calcular el z_score en su version modificada para cada una de los valores del dataframe, asi podemos chequear por outliers.

In [14]:
def z_score_modificado(columna):
    mediana = columna.median()
    MAD = (columna - mediana).abs().median()
    return ((columna - mediana)*0.6745)/MAD

In [15]:
aux = {}
for column in df_train:
    if column not in categoricas and column not in ignorar:
        aux[column] = z_score_modificado(df_train[column])

In [16]:
df_aux = pd.DataFrame(aux)
df_aux

Ahora que ya tenemos el z-score modificado calculado para cada uno de los valores de nuestros features, vamos a visualizar que porcentaje de outliers tiene cada columna, considerando que con un z-score mayor a 3.5 seria considerado un outlier

In [17]:
background_color = 'white'
outliers = pd.DataFrame(columns = ['% Outliers'],data = (df_aux > 3.5).sum()/len(df_aux))
outliers = outliers.sort_values(by=['% Outliers'])
fig = plt.figure(figsize = (20, 60),facecolor=background_color)
gs = fig.add_gridspec(1, 2)
gs.update(wspace = 0.5, hspace = 0.5)
ax0 = fig.add_subplot(gs[0, 0])
for s in ["right", "top","bottom","left"]:
    ax0.spines[s].set_visible(False)
sns.heatmap(outliers.loc[outliers['% Outliers'] > 0],cbar = False,annot = True,fmt =".4%", linewidths = 2,vmax = 1, ax = ax0)
plt.title("Porcentaje de outliers por feature (>3.5)")
plt.show()

Como vemos hay muchas columnas con una enorme cantidad de outliers segun z-score, de todas maneras consideramos que para el caso de estudio estos outliers nos pueden dar informacion muy valiosa ya que en este caso la clase dificil de predecir es justamente la que muestra casos positivos.

Para seguir un poco mas con el analisis de outliers decidimos dividir todos nuestros datos segun la categoria general asociada
* D_* = Delinquency variables
* S_* = Spend variables
* P_* = Payment variables
* B_* = Balance variables
* R_* = Risk variables

Los siguientes graficos no son de mucha ayuda para visualizar valores concretos ya que tenemos muchas features para poder graficar, pero si nos sirven para detectar outliers severos o que se vayan mucho de los valores normales para cada columna

In [18]:
D_columns = [col for col in df_train if col.startswith('D')]
df_train[D_columns].describe()

In [19]:
df_with_d_columns = df_train[D_columns].copy()
sns.set(rc={"figure.figsize":(25, 20)})
fig, axes = plt.subplots(5, 2)
sns.boxplot(data=df_with_d_columns.iloc[:, 0:9], ax=axes[0,0])
sns.boxplot(data=df_with_d_columns.iloc[:, 10:19], ax=axes[0,1])
sns.boxplot(data=df_with_d_columns.iloc[:, 20:29], ax=axes[1,0])
sns.boxplot(data=df_with_d_columns.iloc[:, 30:39],ax=axes[1,1])
sns.boxplot(data=df_with_d_columns.iloc[:, 40:49],ax=axes[2,0])
sns.boxplot(data=df_with_d_columns.iloc[:, 50:59],ax=axes[2,1])
sns.boxplot(data=df_with_d_columns.iloc[:, 60:69],ax=axes[3,0])
sns.boxplot(data=df_with_d_columns.iloc[:, 70:79],ax=axes[3,1])
sns.boxplot(data=df_with_d_columns.iloc[:, 80:89],ax=axes[4,0])
sns.boxplot(data=df_with_d_columns.iloc[:, 90:96],ax=axes[4,1])
plt.suptitle('Delinquency features', fontsize=20, y=0.93)
plt.show()

In [20]:
S_columns = [col for col in df_train if col.startswith('S')]
df_train[S_columns].describe()

In [21]:
df_with_s_columns = df_train[S_columns].copy()
sns.set(rc={"figure.figsize":(15, 5)})
ax = sns.boxplot(data=df_with_s_columns[['S_5', 'S_12', 'S_16', 'S_22', 'S_23', 'S_24', 'S_26']])
plt.title('Spend features')
plt.show()

df_with_s_columns = df_train[S_columns].copy()
sns.set(rc={"figure.figsize":(15, 5)})
ax = sns.boxplot(data=df_with_s_columns[['S_3', 'S_6', 'S_7', 'S_8', 'S_9', 'S_11', 'S_13', 'S_15', 'S_17', 'S_18', 'S_19', 'S_20', 'S_23', 'S_25', 'S_27']])
plt.title('Spend features')
plt.show()



In [22]:
P_columns = [col for col in df_train if col.startswith('P')]
df_train[P_columns].describe()

In [23]:
df_with_p_columns = df_train[P_columns].copy()
sns.set(rc={"figure.figsize":(10, 5)})
ax = sns.boxplot(data=df_with_p_columns)
plt.title('Payment features')
plt.show()

In [24]:
B_columns = [col for col in df_train if col.startswith('B')]
df_train[B_columns].describe()

In [25]:
df_with_b_columns = df_train[B_columns].copy()
sns.set(rc={"figure.figsize":(25, 5)})
ax = sns.boxplot(data=df_with_b_columns)
plt.title('Balance features')
plt.show()

In [26]:
R_columns = [col for col in df_train if col.startswith('R')]
df_train[R_columns].describe()

In [27]:
df_with_r_columns = df_train[R_columns].copy()
sns.set(rc={"figure.figsize":(25, 5)})
ax = sns.boxplot(data=df_with_r_columns)
plt.title('Risk features')
plt.show()

Estas visualizaciones pueden ser de ayuda en caso de que mas adelante en la investigacion determinemos que una cierta feature tiene mucha correlacion o sirve mucho para predecir nuestro target, saber si tiene valores atipicos y actuar en consecuencia para mejorar nuestros modelos.

### Correlacion con la variable target

In [28]:
sns.set(rc={"figure.figsize":(15, 55)})
sns.barplot(x=df_train.corr()["target"].sort_values(ascending=False)[1: -1].values, y=df_train.corr()["target"].sort_values(ascending=False)[1: -1].index, palette="RdBu")
plt.title("Correlacion de features con la variable target")
plt.show()

Como vemos hay variables muy correlacionadas positiva y negativamente y es muy probable que sean utilizadas por el modelo ya que son las que mas cantidad de informacion va a darnos para nuestras predicciones

### Normalización de los datos

No consideramos necesario realizarla ya que en la información que nos brinda la competencia se menciona que las features ya están normalizadas.

### Sobre las variables categoricas

Como vimos tenemos variables categoricas de las cuales queremos ver si podemos sacar mas informacion o correlacion

In [29]:
categoricas = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_68']

In [30]:
plt.figure(figsize=(16, 16))
for index, feature in enumerate(categoricas):
    plt.subplot(4, 3, index+1)
    df_temp = pd.DataFrame(df_train[feature][df_train.target == 0].value_counts(normalize=True).sort_index().rename('count'))
    plt.bar(df_temp.index, df_temp['count'], alpha=0.5, label='target=0')
    df_temp = pd.DataFrame(df_train[feature][df_train.target == 1].value_counts(normalize=True).sort_index().rename('count'))
    plt.bar(df_temp.index, df_temp['count'], alpha=0.5, label='target=1')
    plt.xlabel(feature)
    plt.ylabel('frecuencia')
    plt.legend()
plt.suptitle('Frecuencia del target en features categoricas', fontsize=20, y=0.93)
plt.show()
del df_temp

Esto sirve como primer analisis general en las variables categoricas como vemos algunas features tienen una frecuencia mayor del target 1 que el target 0 y probablemente sea utilizada por nuestros modelos para la prediccion

### Sobre los días de la semana

Basandonos en lo que observamos en una de las notebooks de la competencia, concluímos que podía ser interesante evaluar si hay alguna correlación entre el target y los días de la semana de S_2

In [31]:
!pip install july
import datetime
import july


In [32]:
train = df_train.copy()
labels = df_sample_casos.copy()


train['S_2'] = train['S_2'].apply(datetime.datetime.fromisoformat)
train['N'] = train.groupby('customer_ID')['customer_ID'].transform('count')
train = train.loc[train.N==13] 

tx = train.groupby('S_2')['customer_ID'].count().reset_index(name='N')
ty = train.groupby('S_2')['target'].mean().reset_index(name='target_rate')

In [33]:
july.heatmap(tx['S_2'], tx['N'], title='train count', cmap="golden", colorbar=True, month_grid=True)
july.heatmap(ty['S_2'], ty['target_rate'], title='target rate', cmap="golden", colorbar=True, month_grid=True)
plt.show()

In [34]:
suma_correlacion_por_dia_de_la_semana = [0, 0, 0, 0, 0, 0, 0]
ocurrencias_por_dia_de_la_semana = [0, 0, 0, 0, 0, 0, 0]

for index, row in ty.iterrows():
    suma_correlacion_por_dia_de_la_semana[row['S_2'].weekday()] += row['target_rate']
    ocurrencias_por_dia_de_la_semana[row['S_2'].weekday()] += 1

correlacion_por_dia_de_la_semana = [m/n for m, n in zip(suma_correlacion_por_dia_de_la_semana, ocurrencias_por_dia_de_la_semana)]
correlacion_por_dia_de_la_semana

Observamos que algunos días (por ejemplo los domingos) muestran una correlación minimamente superior a las de otros, pero la correlacion es muy baja para todos respecto a esta variable. Es posible que se termine eliminando ya que el analisis nos dice que no es relevante para la prediccion del target.

## Transformación

Para empezar con la transformacion de nuestros datos vamos a eliminar en un principio las variables que tengan una media de valores NaNs mayor al 70%.

In [35]:
df_train_reduced = df_train.loc[:, df_train.isnull().mean() < 0.7].copy()

Ahora deifinimos nuestras nuevas variables categoricas ya que se eliminaron algunas con gran cantidad de valores faltantes y chequeamos que valores tiene cada una para determinar una manera de rellenarlas.

In [36]:
for variable in categoricas:
    print(df_train_reduced[variable].value_counts(dropna = False))

Vamos a rellenar todas las variables que no sean categoricas con la mediana de todos los valores de cada features. Por otro lado en el caso de la variable D_63 vemos que corresponde a prefijos de paises por lo tanto creamos una nueva categoria que sea "No country" que simbolice que no se tiene el pais de ese cliente. Por otro lado vemos que la feature "D_64" tiene un valor -1 que quizas representa valor faltante o algo de lo que no se tiene data asi que vamos a utilizar ese valor para completar. Por otro lado como dijimos anteriormente el resto de features que contengan el prefijo "D" les asignaremos un valor 0, ya que el NaN podria significar que no contienen registro de delito y por lo tanto no se tiene ese campo lo que seria igual a tener un 0. Por ultimo las demas features seran rellenadas con un valor extra en este caso el 9, esto funcionaria como una categoria extra representando que no existe ese valor, como no sabemos que representan las variables no podemos asignar un valor claro y representativo.

In [37]:
for column in df_train_reduced:
    if column not in categoricas and column not in ignorar:
        df_train_reduced[column].fillna(inplace = True, value = df_train_reduced[column].median())
    elif column == "D_63":
        df_train_reduced[column].fillna(inplace = True, value = "No country")
    elif column == "D_64":
        df_train_reduced[column].fillna(inplace = True, value = '-1')
    elif column.startswith("D"):
        df_train_reduced[column].fillna(inplace = True, value = 0)
    else:
        df_train_reduced[column].fillna(inplace = True, value = 9)

A continuacion nosotros vamos a querer una sola prediccion por cada cliente, cada uno representado por su ID. Los datos previstos hacen referencia a muchas "transacciones" o muchos "casos" para cada cliente asi que vamos a calcular la media (en caso de variables numericas) y la moda (en caso de variables categoricas) entre todos los casos que tiene un mismo cliente, y nos quedaremos con esos valores para representarlo. De esta manera nos quedamos con una representacion por cada cliente y podemos hacer nuestras predicciones. Nos quedamos con la cantidad de veces que estaba repetido para crear una nueva feature con esta caracteristica, quizas sea util para nuestro entrenamiento.

In [38]:
repeticiones = df_train_reduced['customer_ID'].value_counts().sort_index()
repeticiones

In [39]:
categoricas_and_customer = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_68', 'customer_ID']
df_train_cat = df_train_reduced[categoricas_and_customer].copy()
df_train_num = df_train_reduced.drop(columns = categoricas).copy()

df_train_num = df_train_num.groupby("customer_ID").mean()
df_train_cat = df_train_cat.groupby("customer_ID").agg(lambda x: pd.Series.mode(x)[0])

In [40]:
df_train_final = pd.merge(df_train_num, df_train_cat, left_on='customer_ID', right_on='customer_ID')

In [41]:
df_train_final = df_train_final.reset_index()
df_train_final["customer_ID"].value_counts()

In [42]:
df_train_final.head()

In [43]:
df_train_final['repeticiones'] = repeticiones.values

Vamos a realizar un pequeño análisis para ver si valdría la pena quedarnos con la información de cuantas repeticiones tenía cada ID

In [44]:
sns.set(rc={"figure.figsize":(15, 7)})
sns.countplot(data=df_train_final, y="repeticiones" , hue="target")
plt.show()

Como se puede observar, la relación entre casos de default y no default no es la misma para cada cantidad de repeticiones, quizas a los modelos les sirve este feature.

## Conclusiones del analisis

Con todo lo analizado podemos acercarnos a una conclusion acerca de los datos previstos, si bien es complicado sacar conclusiones de las features ya que no tienen una representacion que nos permita ver que significan en la realidad, por eso vamos a realizar algunas suposiciones para poder analizar los datos de una manera mas clara:
- Las "features de Delincuencia" (Delinquency variables) tienen un alto contenido de nulos, de hecho 24 de las 30 variables con mas nulos pertenecen a esta categoria. Asumiendo que refieren a delitos o alguna falta en transacciones con tarjeta podemos decir que los nulos probablemente sean casos en los que las persona no tiene faltas por lo tanto pueden ser variables interesantes en los casos en que la persona tenga alguna falta o sea no pertenezca a ese valor 0. Como vemos en la visualizacion hay variables de esta categoria altamente correlacionadas con nuestra variable target.
- Sabiendo que la variable **P_2** es una de las mas correlacionadas (negativamente) es posible que esta feature perteneciente a la categoria de pagos se relacione con el plazo en el cual se paga la tarjeta y en caso de siempre cumplir con la regla de pagarla a tiempo es altamente probable que la persona no vaya a entrar en este "Default".
- Algunas features presentan outliers un poco severos y que se van de los valores tipicos de las otras entradas. Sabiendo que el target con clase minoritaria es el que buscamos, suponemos que esos outliers son casos fuera de la norma que pueden conducir a un Default en un futuro ya que presentan irregularidades.
- Una suposicion que quizas tendria sentido hacer es que la presencia de valores nulos corresponda a personas que tienen una cuenta creada hace relativamente poco tiempo, quizas todavia no son calculados los valores para las diferentes features cuando todavia no paso un tiempo considerable de uso, esto refuerza reemplazar mucho de los NaNs por un valor 0 o un numero representando otra categoria distinta
- Muchas variables parecen no ser relevantes para determinar el Default de un cliente. Probablemente sean descartadas para el entrenamiento del modelo y necesitemos de menos para obtener la misma cantidad de informacion, este dataset debe ser una recopilacion entera de datos especificos de clientes sin necesidad de relacionarse con el Default.
- No vimos una relevancia grande en las fechas de cada entrada. Es posible que las fechas simplemente sean una fecha por cada transaccion o accion que emite el cliente y no tengan especial relevancia para un suceso en particular, es un movimiento mas en esa cuenta.

### Metricas a utilizar

Nos decidimos por f1 y ROC-AUC

# Reduccion de dimensionalidad (PCA)

Como nuestros datos tienen muchisimas features y eso va a imposibilitar el entrenamiento de los modelos en un tiempo razonable vamos a aplicar una reduccion de dimensionalidad.

In [45]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [46]:
df_train = df_train_final.drop(columns = ["customer_ID", "target"])
pd.get_dummies(data = df_train, columns = ["D_63", "D_64"])
df_train.drop(columns = ["D_63", "D_64"], inplace = True)
df_train

In [47]:
pca = PCA(svd_solver = "randomized", random_state = 42)

In [48]:
pca.fit(df_train)

In [49]:
pca.components_

In [50]:
fig = plt.figure(figsize = (10,6))
var_cumu = np.cumsum(pca.explained_variance_ratio_) * 100
k = np.argmax(var_cumu > 95)
plt.plot(var_cumu)
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(x=k, color="k", linestyle="--")
plt.axhline(y=95, color="r", linestyle="--")
plt.show()

In [51]:
fig = plt.figure(figsize = (10,6))
plt.plot(pca.explained_variance_ratio_[0:30],'o-',markersize=2)
plt.xlabel('Componente')
plt.ylabel('Varianza explicada')
plt.show()

Con los datos calculados y las visualizaciones hechas podemos ver que con 20 componentes de PCA ya tenemos el 95% de la varianza explicada por lo tanto podemos aplicar PCA con 20 componentes y contendria la mayoria de la informacion que nuestros datos nos estan diciendo

In [52]:
final_pca = PCA(n_components = 20)

In [53]:
df_pca_transform = pd.DataFrame(final_pca.fit_transform(df_train))

In [54]:
df_pca_transform["target"] = df_train_final["target"]

In [55]:
df_pca_transform.head()

In [56]:
df_pca_transform.to_csv('df_train_pca.csv', sep=";")

## Modelos

Notebook con los modelos: https://www.kaggle.com/code/ignacioavecilla/tp3-datos-modelos

## Conclusiones finales

Los modelos con los que mejores resultados obtuvimos fueron la red neuronal y el ensamble en cascada, ambos con resultados bastante similares. Si lo que se quiere es aumentar la precisión con la que se detectan los casos de default, probablemente el ensamble sea la mejor opción. El problema es que quedan muchos casos sin clasificar, casi un 40%, que en este caso decidimos volver a pasarselos a la red ya que es nuestro mejor modelo.

Concluímos que es posible detectar si un cliente va a dejar de pagar en la mayoría de los casos, pero teniendo tan poca información sobre el significado de los datos es complejo realizar el tratamiento necesario de los mismos para obtener el mejor resultado posible.  

Información como antecendentes penales o historial crediticio podrían ser muy útiles para lograr mejores predicciones.

Si se tuviera conocimiento más certero sobre el significado de los datos, probablemente la intervención humana tendría un peso relevante en un sistema en producción, pudiendo investigar minusiosamente casos particulares. 